In [ ]:
%pip install pandas
%pip install openpyxl

import pandas as pd
from pathlib import Path
import requests
import os


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Downloading XLS files from council website

In [ ]:


# Create list of target URLs to download
classified_urls_df = pd.read_csv('../wayback/url_classified.csv')
classified_urls_df['target_url'] = classified_urls_df['wayback_download_url'].str.split('id_/').str[1]

list_new_businesses = classified_urls_df[classified_urls_df['category'] == 'new_businesses']['target_url'].dropna().unique()
list_accounts_no_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_no_relief']['target_url'].dropna().unique()
list_account_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_relief']['target_url'].dropna().unique()
list_accouts_closed = classified_urls_df[classified_urls_df['category'] == 'accounts_closed']['target_url'].dropna().unique()
list_account_current_in_credit = classified_urls_df[classified_urls_df['category'] == 'accounts_current_in_credit']['target_url'].dropna().unique()

# Download raw files to local
save_dirs = ['new_businesses', 'accounts_no_relief', 'accounts_relief', 'accounts_closed', 'accounts_current_in_credit']

config = {
    'new_businesses': {
        'urls': list_new_businesses,
        'save_dir': 'new_businesses',
    },
    'accounts_no_relief': {
        'urls': list_accounts_no_relief,
        'save_dir': 'accounts_no_relief',
    },
    'accounts_relief': {
        'urls': list_account_relief,
        'save_dir': 'accounts_relief',
    },
    'accounts_closed': {
        'urls': list_accouts_closed,
        'save_dir': 'accounts_closed',
    },
    'accounts_current_in_credit': {
        'urls': list_account_current_in_credit,
        'save_dir': 'accounts_current_in_credit',
    },
}

for save_dir in save_dirs:
    os.makedirs('raw_files/' + save_dir, exist_ok=True)

for category, config in config.items():
    for url in config['urls']:
        raw_file_name = url.split('/')[-1]
        # Stream the content and write it to disk
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # raises exception if download failed
            with open('raw_files/' + config['save_dir'] + '/' + raw_file_name, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)


# Sanitization

### New businesses

In [ ]:


config = {
    'new_businesses': {
        'folder_path': 'raw_files/new_businesses',
        'column_mapping': {
            "Current Analysis Code Description": "current_analysis_code_description",
            "Primary Liable party name": "primary_liable_party_name",
            "Property Reference Number": "property_reference_number",
            "Full Property Address": "full_property_address",
            "Liable Responsibility Start Date": "liable_responsibility_start_date",
            "Current Rateable Value": "current_rateable_value",
        },
        'desired_order': [
            "current_analysis_code_description",
            "primary_liable_party_name",
            "property_reference_number",
            "full_property_address",
            "liable_responsibility_start_date",
            "current_rateable_value", 
            "source_file"
        ]
    },
    'accounts_closed': {
        'folder_path': 'raw_files/accounts_closed',
        'column_mapping': {
            'Property Reference Number' : 'property_reference_number',
            'Primary Liable party name' : 'primary_liable_party_name',
            'Full Property Address' : 'full_property_address',
            'Account Start date' : 'account_start_date',
            'Account End date' : 'account_end_date',
            'Outstanding Debt' : 'outstanding_debt'
        },
        'desired_order': [
           'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'account_start_date',
            'account_end_date',
            'outstanding_debt'
        ]
    },
    'accounts_relief': {
        'folder_path': 'raw_files/accounts_relief',
        'column_mapping': {
            'Property Reference Number' : 'property_reference_number',
            'Primary Liable party name' : 'primary_liable_party_name',
            'Full Property Address' : 'full_property_address',
            'Liable Responsibility Start Date' : 'liable_responsibility_start_date',
            'Current Rateable Value' : 'current_rateable_value',
            'Current Relief Type' : 'current_relief_type',
            'Current Relief Type Description' : 'current_relief_type_description',
            'Current Relief Award Start Date' : 'current_relief_award_start_date',
            'Current Relief Award Perc Awarded' : 'current_relief_award_perc_awarded'
        },
        'desired_order': [
            'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'liable_responsibility_start_date',
            'current_rateable_value',
            'current_relief_type',
            'current_relief_type_description',
            'current_relief_award_start_date',
            'current_relief_award_perc_awarded'
            
        ]
    },
    "accounts_no_relief": {
        'folder_path': 'raw_files/accounts_no_relief',
        'column_mapping': {
            'Property Reference Number': 'property_reference_number',
            'Primary Liable party name': 'primary_liable_party_name',
            'Full Property Address': 'full_property_address',
            'Liable Responsibility Start Date': 'liable_responsibility_start_date',
            'Current Rateable Value': 'current_rateable_value',
            'Billing Period': 'billing_period',
            'A/C Summary Current Liability': 'a_c_summary_current_liability'
        },
        'desired_order': [
            'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'liable_responsibility_start_date',
            'current_rateable_value',
            'billing_period',
            'a_c_summary_current_liability'
        ]
    }
}

def concat_dataframes(folder_path, column_mapping, desired_order):
    dfs = []
    for file_path in Path(folder_path).glob('*.xlsx'):
        print(f'Processing {file_path}')
        # Load Excel file into pandas
        df = pd.read_excel(file_path, sheet_name=0)  # sheet_name=0 loads first sheet
        # Rename columns using the mapping
        df = df.rename(columns=column_mapping)
        # Add filename column for reference
        df['source_file'] = file_path.name

        existing_columns = [col for col in desired_order if col in df.columns]
        df = df[existing_columns]
        # Store the dataframe
        dfs.append(df)

    result_df = pd.concat(dfs, ignore_index=True)
    return result_df



new_businesses_consolidated_df = concat_dataframes(
    'raw_files/new_businesses',
    config['new_businesses']['column_mapping'],
    config['new_businesses']['desired_order']
)

businesses_closed_consolidated_df = concat_dataframes(
    'raw_files/accounts_closed',
    config['accounts_closed']['column_mapping'],
    config['accounts_closed']['desired_order']
)

accounts_relief_consolidated_df = concat_dataframes(
    'raw_files/accounts_relief',
    config['accounts_relief']['column_mapping'],
    config['accounts_relief']['desired_order']
)

accounts_no_relief_consolidated_df = concat_dataframes(
    'raw_files/accounts_no_relief',
    config['accounts_no_relief']['column_mapping'],
    config['accounts_no_relief']['desired_order']
)










Processing raw_files/new_businesses/NNDR%20New%20Businesses%20from%20June%20to%20October%202023.xlsx
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20October20%20to%20January21.xlsx
Processing raw_files/new_businesses/New%20Businesses%20July-Sept20_0.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20October%202021%20to%20March%202022.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20April22%20to%20August22.xlsx
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20Nov23%20to%20March24.xlsx
Processing raw_files/new_businesses/New%20Businesses%20from%20July%20to%20September21.xlsx
Processing raw_files/new_businesses/NNDR%20New%20Businesses%20from%20September%202022%20to%20January%202023.xlsx
Processing raw_files/new_businesses/NNDR%20New%20Businesses_February%20to%20May2023.xlsx
Processing raw_files/new_businesses/New%20businesses%20from%20February%20to%20June21.xlsx
Processing raw_files/accounts_closed/NNDR%20Closed%20Acco

In [ ]:
df0 = pd.read_excel("raw_files/accounts_no_relief/NNDR%20Accounts%20With%20No%20Relief_April22.xlsx", sheet_name=0)
print(df0.columns)
# df0 = df0.rename(columns=column_mapping)
# df0['source_file'] = "abc"
# existing_columns = [col for col in desired_order if col in df0.columns]
# df0 = df0[existing_columns]

print(df0)




Index(['Property Reference Number', 'Primary Liable party name',
       'Full Property Address', 'Liable Responsibility Start Date',
       'Current Rateable Value', '2017 NDR - Analysis code desc',
       'Billing Period', 'A/C Summary Current Liability'],
      dtype='object')
     Property Reference Number             Primary Liable party name  \
0             NN11001775998107            St Albans District Council   
1             NN11001014034191                             RATEPAYER   
2             NN11001017056015                             RATEPAYER   
3             NN11001017997009    Albert Street Car Park Association   
4             NN1100102106801B         Phillips Funeral Services Ltd   
...                        ...                                   ...   
3965          NN1100141611306B                             RATEPAYER   
3966          NN11001522921031  Principled Offsite Logistics Limited   
3967          NN11004005815015                        Biz-Space  Ltd   
